<a href="https://colab.research.google.com/github/mahdi-tavakoli-anodyneGROUP/Collage-AI-4/blob/main/train1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BhJ63IemG3ACRXXxUyzC")
project = rf.workspace("dental-decay").project("alpha-1-60hzc")
version = project.version(3)
dataset = version.download("yolov8")





loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os
import yaml
import cv2
import numpy as np
! pip install ultralytics
from ultralytics import YOLO

In [ ]:
train_params = {
    "epochs": 10,
    'imgsz': 640,
    'batch': 8,
    'workers': 2,
    'device': 'cpu',
    'cls':1.0, #vazn class
    'patience':100,
    'optimizer': 'Adam',
    'lr0': 0.001,
}


In [ ]:
def preprocess_image(image_path):
    #tasvir khondam
    image = cv2.imread(image_path)
    #vojod tasvir
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return None
    #labe tiz kardam
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    #filter tiz labe be tasvir dadam
    sharpened = cv2.filter2D(image, -1, kernel)
    #tashkhis labe
    edges = cv2.Canny(sharpened , 100 , 200)
    return edges

#masir tasvir
dataset_path = "/content/alpha-1-3"
output_path = "/content/alpha-1-3/train/images"
#zakhire khorogi
if not os.path.exists(output_path):
    os.makedirs(output_path)

# pardazesh tasvir
for filename in os.listdir(dataset_path):
    image_path = os.path.join(dataset_path, filename)
    if os.path.isfile(image_path) and (filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg')):
        processed_image = preprocess_image(image_path)
        if processed_image is not None:
            cv2.imwrite(os.path.join(output_path, filename), processed_image)
#yaml
data_yaml_path = os.path.join(dataset_path, '/content/alpha-1-3/data.yaml')

#moshakhas kardan masir train , val , test
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
data['train'] = '/content/alpha-1-3/train/images'
data['val'] = '/content/alpha-1-3/valid/images'
data['test'] = '/content/alpha-1-3/test/images'

with open(data_yaml_path, 'w') as f:
    yaml.safe_dump(data, f)

# train kardan
# model = YOLO('yolov8n.pt')
model=YOLO('yolov8s-seg.pt')
try:
    model.train(data=data_yaml_path, **train_params)
except Exception as e:
    print(f"Training error: {e}")


Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/alpha-1-3/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=2, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/alpha-1-3/train/labels.cache... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
val: Scanning /content/alpha-1-3/valid/labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]


Plotting labels to runs/segment/train3/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 72 weight(decay=0.0), 83 weight(decay=0.0005), 82 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10         0G      1.314       2.03      1.819      1.073          6        640: 100%|██████████| 14/14 [03:35<00:00, 15.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:13<00:00,  6.74s/it]

                   all         31         31          1      0.999      0.995      0.608          1      0.999      0.995      0.526



10 epochs completed in 0.651 hours.
Optimizer stripped from runs/segment/train3/weights/last.pt, 21.3MB
Optimizer stripped from runs/segment/train3/weights/best.pt, 21.3MB

Validating runs/segment/train3/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (AMD EPYC 7B12)
YOLOv8s-seg summary (fused): 213 layers, 10,482,067 parameters, 0 gradients, 37.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:13<00:00,  6.56s/it]


                   all         31         31          1      0.999      0.995      0.608          1      0.999      0.995      0.526
Speed: 2.0ms preprocess, 375.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train3
